In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import xlrd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC

In [2]:
xlrd.xlsx.ensure_elementtree_imported(False, None)
xlrd.xlsx.Element_has_iter = True

data = pd.read_csv("train.csv")
target = pd.read_excel("sample_submission.xlsx")

<ipython-input-2-f27a38a35fe7>:5: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  target = pd.read_excel("sample_submission.xlsx")


In [3]:
data.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... ScreenPorch PoolArea PoolQC Fence MiscFeature  \
0         Lvl    AllPub  ...           0        0    NaN   NaN         NaN   
1         Lvl    AllPub  ...           0        0    NaN   NaN         NaN   
2         Lvl    AllPub  ...           0        0    NaN   NaN         NaN   
3         Lvl    AllPub  ...           0        0    NaN   NaN         NaN   
4         Lvl    AllPub  ...           0        0    NaN   NaN         NaN   

  MiscVal MoSold  YrSold  SaleType  SaleCondition  
0       0      2    2008        WD         Normal  
1       0      5    2007        WD         Normal  
2       0      9    2008        WD         Normal  
3       0      2    2006        WD        Abnorml  
4       0     12    2008        WD         Normal  

[5 rows x 80 columns]

In [4]:
target.head()

Id  Actual_SalePrice  Linear_Regression_Predict_SalePrice  \
0   1            208500                                  NaN   
1   2            181500                                  NaN   
2   3            223500                                  NaN   
3   4            140000                                  NaN   
4   5            250000                                  NaN   

   Gaussian_Process_Predict_SalePrice  Random_Forest_Predict_SalePrice  \
0                                 NaN                              NaN   
1                                 NaN                              NaN   
2                                 NaN                              NaN   
3                                 NaN                              NaN   
4                                 NaN                              NaN   

   XGBoost_Predict_SalePrice  Stocked_Predict_SalePrice  
0                        NaN                        NaN  
1                        NaN                        NaN  
2                        NaN                        NaN  
3                        NaN                        NaN  
4                        NaN                        NaN

In [5]:
# Creating the target column in dataset
targets = target['Actual_SalePrice'].tolist()
data["SalePrice"] = targets

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [7]:
# Dropping columns with mostly NAN values / useless columns

data = data.drop(["FireplaceQu","Alley","PoolQC","Fence","MiscFeature"],axis = 1)
data = data.dropna()

In [8]:
# Correlation matrix

corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

With this we pick 20 columns with high correlation to the SalePrice and remove the rest

In [9]:
SaleCorr = corr["SalePrice"]

SaleCorr = SaleCorr.abs().sort_values(ascending=False)
print(SaleCorr[0:21])
SaleCorr = SaleCorr.reset_index()

SalePrice       1.000000
OverallQual     0.795437
GrLivArea       0.707481
GarageCars      0.652103
GarageArea      0.620772
TotalBsmtSF     0.617741
1stFlrSF        0.617692
FullBath        0.578299
TotRmsAbvGrd    0.560521
YearBuilt       0.523434
YearRemodAdd    0.519806
GarageYrBlt     0.502248
MasVnrArea      0.485409
Fireplaces      0.458182
BsmtFinSF1      0.378678
LotFrontage     0.343978
OpenPorchSF     0.338600
WoodDeckSF      0.330286
2ndFlrSF        0.302569
LotArea         0.302268
HalfBath        0.259469
Name: SalePrice, dtype: float64


In [10]:
colList = []
for i in SaleCorr["index"][0:21]:
    colList.append(i)
    
colList.append("Id")
    
data = data[colList]

In [11]:
data.head()

SalePrice  OverallQual  GrLivArea  GarageCars  GarageArea  TotalBsmtSF  \
0     208500            7       1710           2         548          856   
1     181500            6       1262           2         460         1262   
2     223500            7       1786           2         608          920   
3     140000            7       1717           3         642          756   
4     250000            8       2198           3         836         1145   

   1stFlrSF  FullBath  TotRmsAbvGrd  YearBuilt  ...  MasVnrArea  Fireplaces  \
0       856         2             8       2003  ...       196.0           0   
1      1262         2             6       1976  ...         0.0           1   
2       920         2             6       2001  ...       162.0           1   
3       961         1             7       1915  ...         0.0           1   
4      1145         2             9       2000  ...       350.0           1   

   BsmtFinSF1  LotFrontage  OpenPorchSF  WoodDeckSF  2ndFlrSF  LotArea  \
0         706         65.0           61           0       854     8450   
1         978         80.0            0         298         0     9600   
2         486         68.0           42           0       866    11250   
3         216         60.0           35           0       756     9550   
4         655         84.0           84         192      1053    14260   

   HalfBath  Id  
0         1   1  
1         0   2  
2         1   3  
3         0   4  
4         1   5  

[5 rows x 22 columns]

In [12]:
Ids = data["Id"].tolist()
data = data.drop("Id",axis=1)

X = data.drop("SalePrice",axis = 1)
y = data["SalePrice"]

lreg = LinearRegression()
lreg.fit(X,y)

mean_squared_error(y,lreg.predict(X))

1472412858.9892142

In [15]:
y_pred = lreg.predict(X)

y_pred

array([220580.18203679, 183048.70074527, 222427.72722909, ...,
       227528.4020458 , 119550.95270262, 142310.56816107])